# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [43]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'

r = requests.get(url)

data = r.json()

df = pd.DataFrame(data['items'])

df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,6833,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.000440/2008-75,2008-04-11T00:00:00Z,Operação contratual interna,Multissetorial,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,15000000.0,1,0,18/04/2008
1,13271,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001999/2008-12,2009-07-01T00:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,39500000.0,1,0,07/07/2009
2,5918,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000233/2012-05,2012-04-16T00:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,319675000.0,1,0,11/03/2016
3,8556,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000797/2010-78,2011-06-30T00:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,112000000.0,1,0,04/07/2011
4,7080,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000492/2012-28,2012-04-16T00:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,300000000.0,1,0,03/09/2012


In [54]:
# 2) Seu código aqui
print(df['status'].nunique())
print(df['status'].unique())
print(df.status.value_counts())

4
['Deferido' 'Encaminhado à PGFN com manifestação técnica favorável'
 'Arquivado a pedido' 'Arquivado por decurso de prazo']
status
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Arquivado por decurso de prazo                            1
Name: count, dtype: int64


In [79]:
# 3) Seu código aqui
ano_status = df['data_status'].str.split(pat="/").str[2]
ano_status.head()

0    2008
1    2009
2    2016
3    2011
4    2012
Name: data_status, dtype: object

In [80]:
#4
ano_status.value_counts()

data_status
2012    11
2009     9
2013     7
2011     6
2016     4
2002     4
2014     4
2010     3
2008     2
2006     2
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [161]:
#1) Seu código aqui
url1 = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'

def consulta (uf,tipo_interessado):
    r = requests.get(url1)
    r.status_code
    data = r.json()
    df = pd.DataFrame(data['items'])
    
    consulta = (df['uf'] == uf) & (df['tipo_interessado'] == tipo_interessado)
    
    df_consulta = df[consulta]
    return df_consulta

In [173]:
# 2) Seu código aqui

uf = 'MG'
tipo_interessado = 'Município'

resultado = consulta(uf,tipo_interessado)
resultado_mg = resultado[resultado['status'] == 'Arquivado por decurso de prazo']
resultado_mg


#Quando eu colocava Estado não aparecia nenhum dado, só o cabeçalho

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
17,12282,Município,Córrego Fundo,3119955,MG,None,Arquivado por decurso de prazo,17944.001648/2011-15,2012-04-16T00:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,1400000.00,0,0,21/05/2012
85,12978,Município,Passabém,3147501,MG,None,Arquivado por decurso de prazo,17944.001870/2013-71,2013-11-19T00:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,400000.00,0,0,17/10/2014
94,5278,Município,Santo Antônio do Grama,3160108,MG,None,Arquivado por decurso de prazo,17944.000076/2012-20,2012-10-23T00:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,1000000.00,0,0,31/10/2012
172,12203,Município,Lambari,3137809,MG,None,Arquivado por decurso de prazo,17944.001626/2011-47,2012-11-05T00:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,2755000.00,0,0,19/11/2012
197,21189,Município,Perdões,3149903,MG,00000.000000/2025-42,Arquivado por decurso de prazo,17944.000110/2016-90,2016-01-26T00:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,800000.00,0,0,03/10/2016
201,21228,Município,Ituiutaba,3134202,MG,00000.000000/2025-45,Arquivado por decurso de prazo,17944.000130/2016-61,2016-05-17T00:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,1200000.00,0,0,07/10/2016
207,21871,Município,Pompéu,3152006,MG,00000.000000/2028-79,Arquivado por decurso de prazo,17944.000310/2016-42,2016-04-18T00:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,200000.00,0,0,10/10/2016
251,6050,Município,Divisópolis,3122454,MG,None,Arquivado por decurso de prazo,17944.000265/2011-11,2011-03-11T00:00:00Z,Operação contratual interna,Caminho da Escola 2010,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,458000.00,0,0,18/01/2013
510,22246,Município,Capelinha,3112307,MG,00000.000000/2030-66,Arquivado por decurso de prazo,17944.000480/2016-27,2016-04-04T00:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,800000.00,0,0,13/10/2016
784,5025,Município,Santa Rita do Itueto,3159506,MG,None,Arquivado por decurso de prazo,17944.000025/2014-60,2013-12-10T00:00:00Z,Operação contratual interna,"Aquisição de terrenos, imóveis e/ou construção...",Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,300000.00,0,0,16/10/2014


In [181]:
# 3) Seu código aqui

uf = 'BA'
tipo_interessado = 'Município'

resultado = consulta(uf,tipo_interessado)
resultado_ba = resultado[resultado['status'] == 'Deferido']
resultado_ba['interessado'].value_counts()

interessado
Salvador                  3
Vitória da Conquista      3
Luís Eduardo Magalhães    3
Bom Jesus da Lapa         3
Alagoinhas                2
                         ..
Sapeaçu                   1
Ibicuí                    1
Nazaré                    1
Wanderley                 1
Barra do Rocha            1
Name: count, Length: 80, dtype: int64

In [184]:
# 4) Seu código aqui

uf = 'BA'
tipo_interessado = 'Estado'

resultado_bahia = consulta(uf,tipo_interessado)

resultado_bahia.to_csv()

',id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status\n249,18550,Estado,Bahia,29,BA,,Deferido,19407.000048/2006-77,2006-07-11T00:00:00Z,Operação contratual interna,Segurança pública,Instituição Financeira Nacional,Banco do Brasil S/A,Dólar dos EUA,457205.74,1,0,20/07/2006\n334,18583,Estado,Bahia,29,BA,,Deferido,19407.000057/2002-34,2002-06-17T00:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1303036.61,1,0,05/07/2002\n357,18441,Estado,Bahia,29,BA,,Deferido,19407.000023/2005-92,2006-01-16T00:00:00Z,Operação contratual interna,Desenvolvimento sustentável,Instituição Financeira Nacional,Banco do Nordeste do Brasil S/A,Dólar dos EUA,15000000.0,1,0,21/02/2006\n545,69633,Estado,Bahia,29,BA,PVL02.001281/2024-83,Encaminhado à PGFN com manifestação técnica favorável,17944.002726/202